# Sistema de Recomendações - Economiza+ MVP

**Sprint 2 - Dia 8**  
**Objetivo:** Criar sistema de recomendações com 2 regras por cluster

**Hipótese H1:** Recomendações geram economia de 15-20%

**Restrição:** Exatamente 2 regras por cluster (8 regras no total)

## 1. Setup e Carregamento de Dados

In [2]:
import pandas as pd
import numpy as np
import json
import os

# Configurações
pd.set_option('display.float_format', lambda x: f'R$ {x:,.2f}' if abs(x) > 1 else f'{x:.2%}')
np.random.seed(42)

In [3]:
# Carregar dados
usuarios_clustered = pd.read_csv('../data/processed/usuarios_clustered.csv')
transacoes = pd.read_csv('../data/raw/transacoes.csv')

print(f"Usuários: {len(usuarios_clustered)}")
print(f"Transações: {len(transacoes)}")
print(f"\nDistribuição de Clusters:")
print(usuarios_clustered['cluster'].value_counts().sort_index())

Usuários: 500
Transações: 194231

Distribuição de Clusters:
cluster
0     59
1    196
2    167
3     78
Name: count, dtype: int64


## 2. Análise de Gastos por Categoria e Cluster

Vamos analisar os gastos médios por categoria para cada cluster, focando nas categorias não essenciais.

In [4]:
# Filtrar apenas gastos (excluir Renda)
gastos = transacoes[transacoes['categoria'] != 'Renda'].copy()

# Merge com cluster
gastos = gastos.merge(usuarios_clustered[['user_id', 'cluster']], on='user_id')

print(f"Total de transações de gasto: {len(gastos)}")

Total de transações de gasto: 191231


In [5]:
# Definir categorias não essenciais (maior potencial de economia)
CATEGORIAS_NAO_ESSENCIAIS = [
    'Alimentacao_Fora',
    'Vestuario',
    'Lazer',
    'Outros'
]

# Categorias essenciais (menor flexibilidade)
CATEGORIAS_ESSENCIAIS = [
    'Alimentacao_Casa',
    'Habitacao_Aluguel',
    'Habitacao_Contas',
    'Transporte',
    'Saude',
    'Educacao',
    'Telecomunicacoes',
    'Higiene_Limpeza'
]

print("Categorias não essenciais (foco das recomendações):")
for cat in CATEGORIAS_NAO_ESSENCIAIS:
    print(f"  - {cat}")

Categorias não essenciais (foco das recomendações):
  - Alimentacao_Fora
  - Vestuario
  - Lazer
  - Outros


In [6]:
# Calcular gasto mensal médio por usuário por categoria
gasto_mensal = gastos.groupby(['user_id', 'cluster', 'categoria', 'mes'])['valor'].sum().reset_index()
gasto_medio_usuario = gasto_mensal.groupby(['user_id', 'cluster', 'categoria'])['valor'].mean().reset_index()

# Agregar por cluster e categoria
gasto_por_cluster_categoria = gasto_medio_usuario.groupby(['cluster', 'categoria'])['valor'].agg(['mean', 'std', 'count']).reset_index()
gasto_por_cluster_categoria.columns = ['cluster', 'categoria', 'gasto_medio', 'gasto_std', 'num_usuarios']

print("Gasto médio mensal por cluster e categoria calculado!")

Gasto médio mensal por cluster e categoria calculado!


In [7]:
# Nomes dos clusters
NOMES_CLUSTERS = {
    0: 'Endividados Severos',
    1: 'Em Alerta',
    2: 'Endividados Moderados',
    3: 'Poupadores'
}

# Filtrar apenas categorias não essenciais e criar pivot
gastos_nao_essenciais = gasto_por_cluster_categoria[
    gasto_por_cluster_categoria['categoria'].isin(CATEGORIAS_NAO_ESSENCIAIS)
].copy()

# Pivot para visualização
pivot_gastos = gastos_nao_essenciais.pivot(index='categoria', columns='cluster', values='gasto_medio')
pivot_gastos.columns = [NOMES_CLUSTERS[c] for c in pivot_gastos.columns]

print("\n=== GASTO MÉDIO MENSAL POR CATEGORIA NÃO ESSENCIAL (R$/mês) ===")
print(pivot_gastos.round(2).to_string())


=== GASTO MÉDIO MENSAL POR CATEGORIA NÃO ESSENCIAL (R$/mês) ===
                  Endividados Severos  Em Alerta  Endividados Moderados  Poupadores
categoria                                                                          
Alimentacao_Fora            R$ 619.91  R$ 306.75              R$ 432.53   R$ 472.94
Lazer                       R$ 232.53  R$ 113.96              R$ 167.86   R$ 170.53
Outros                      R$ 155.41   R$ 75.36              R$ 103.98   R$ 116.82
Vestuario                   R$ 294.00  R$ 154.14              R$ 207.73   R$ 212.20


In [8]:
# Identificar top categorias de gasto por cluster
print("\n=== TOP 2 CATEGORIAS NÃO ESSENCIAIS POR CLUSTER ===")

for cluster_id in sorted(gastos_nao_essenciais['cluster'].unique()):
    cluster_data = gastos_nao_essenciais[gastos_nao_essenciais['cluster'] == cluster_id]
    top2 = cluster_data.nlargest(2, 'gasto_medio')
    
    print(f"\nCluster {cluster_id} - {NOMES_CLUSTERS[cluster_id]}:")
    for _, row in top2.iterrows():
        print(f"  {row['categoria']}: R$ {row['gasto_medio']:.2f}/mês")


=== TOP 2 CATEGORIAS NÃO ESSENCIAIS POR CLUSTER ===

Cluster 0 - Endividados Severos:
  Alimentacao_Fora: R$ 619.91/mês
  Vestuario: R$ 294.00/mês

Cluster 1 - Em Alerta:
  Alimentacao_Fora: R$ 306.75/mês
  Vestuario: R$ 154.14/mês

Cluster 2 - Endividados Moderados:
  Alimentacao_Fora: R$ 432.53/mês
  Vestuario: R$ 207.73/mês

Cluster 3 - Poupadores:
  Alimentacao_Fora: R$ 472.94/mês
  Vestuario: R$ 212.20/mês


## 3. Definição das Regras de Recomendação

Cada cluster terá exatamente 2 regras baseadas nas categorias com maior potencial de economia.

In [9]:
# Definir regras de recomendação
# Estrutura: categoria, percentual_reducao, mensagem

REGRAS_RECOMENDACAO = {
    0: {  # Endividados Severos - cortes drásticos
        'nome': 'Endividados Severos',
        'prioridade': 'CRÍTICA',
        'regras': [
            {
                'id': 'R0_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.70,
                'titulo': 'Cortar refeições fora de casa',
                'mensagem': 'Reduza drasticamente refeições fora de casa. Priorize marmitas e cozinhar em casa.',
                'dica': 'Planeje cardápio semanal e faça compras com lista'
            },
            {
                'id': 'R0_2',
                'categoria': 'Vestuario',
                'acao': 'eliminacao',
                'percentual': 0.90,
                'titulo': 'Suspender compras de vestuário',
                'mensagem': 'Suspenda compras de roupas não essenciais por 3 meses.',
                'dica': 'Revise o guarda-roupa e conserte peças antes de comprar novas'
            }
        ]
    },
    1: {  # Em Alerta - reduções moderadas
        'nome': 'Em Alerta',
        'prioridade': 'MODERADA',
        'regras': [
            {
                'id': 'R1_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.40,
                'titulo': 'Reduzir refeições fora de casa',
                'mensagem': 'Limite refeições fora a 1-2x por semana.',
                'dica': 'Leve marmita para o trabalho pelo menos 3x por semana'
            },
            {
                'id': 'R1_2',
                'categoria': 'Lazer',
                'acao': 'limite',
                'percentual': 0.35,
                'titulo': 'Estabelecer teto para lazer',
                'mensagem': 'Defina um limite mensal para gastos com lazer.',
                'dica': 'Busque alternativas gratuitas: parques, eventos culturais, bibliotecas'
            }
        ]
    },
    2: {  # Endividados Moderados - cortes significativos
        'nome': 'Endividados Moderados',
        'prioridade': 'ALTA',
        'regras': [
            {
                'id': 'R2_1',
                'categoria': 'Alimentacao_Fora',
                'acao': 'reducao',
                'percentual': 0.50,
                'titulo': 'Reduzir significativamente refeições fora',
                'mensagem': 'Reduza refeições fora de casa pela metade.',
                'dica': 'Cozinhe em maior quantidade no fim de semana para a semana toda'
            },
            {
                'id': 'R2_2',
                'categoria': 'Vestuario',
                'acao': 'reducao',
                'percentual': 0.50,
                'titulo': 'Cortar gastos com vestuário',
                'mensagem': 'Reduza compras de roupas pela metade.',
                'dica': 'Compre apenas itens essenciais e aproveite promoções'
            }
        ]
    },
    3: {  # Poupadores - otimizações
        'nome': 'Poupadores',
        'prioridade': 'BAIXA',
        'regras': [
            {
                'id': 'R3_1',
                'categoria': 'Transporte',
                'acao': 'otimizacao',
                'percentual': 0.15,
                'titulo': 'Otimizar gastos com transporte',
                'mensagem': 'Avalie alternativas de transporte mais econômicas.',
                'dica': 'Considere caronas, transporte público ou bicicleta para trajetos curtos'
            },
            {
                'id': 'R3_2',
                'categoria': 'Telecomunicacoes',
                'acao': 'revisao',
                'percentual': 0.20,
                'titulo': 'Revisar planos e assinaturas',
                'mensagem': 'Revise planos de celular, internet e streaming.',
                'dica': 'Cancele assinaturas não utilizadas e negocie valores com operadoras'
            }
        ]
    }
}

print("Regras de recomendação definidas!")
print(f"Total: {sum(len(v['regras']) for v in REGRAS_RECOMENDACAO.values())} regras (2 por cluster)")

Regras de recomendação definidas!
Total: 8 regras (2 por cluster)


In [10]:
# Exibir regras formatadas
print("\n" + "="*70)
print("REGRAS DE RECOMENDAÇÃO POR CLUSTER")
print("="*70)

for cluster_id, cluster_data in REGRAS_RECOMENDACAO.items():
    print(f"\n### Cluster {cluster_id}: {cluster_data['nome']} (Prioridade: {cluster_data['prioridade']})")
    print("-"*50)
    
    for i, regra in enumerate(cluster_data['regras'], 1):
        print(f"\n  Regra {i}: {regra['titulo']}")
        print(f"  Categoria: {regra['categoria']}")
        print(f"  Ação: {regra['acao']} ({regra['percentual']*100:.0f}%)")
        print(f"  Mensagem: {regra['mensagem']}")
        print(f"  Dica: {regra['dica']}")


REGRAS DE RECOMENDAÇÃO POR CLUSTER

### Cluster 0: Endividados Severos (Prioridade: CRÍTICA)
--------------------------------------------------

  Regra 1: Cortar refeições fora de casa
  Categoria: Alimentacao_Fora
  Ação: reducao (70%)
  Mensagem: Reduza drasticamente refeições fora de casa. Priorize marmitas e cozinhar em casa.
  Dica: Planeje cardápio semanal e faça compras com lista

  Regra 2: Suspender compras de vestuário
  Categoria: Vestuario
  Ação: eliminacao (90%)
  Mensagem: Suspenda compras de roupas não essenciais por 3 meses.
  Dica: Revise o guarda-roupa e conserte peças antes de comprar novas

### Cluster 1: Em Alerta (Prioridade: MODERADA)
--------------------------------------------------

  Regra 1: Reduzir refeições fora de casa
  Categoria: Alimentacao_Fora
  Ação: reducao (40%)
  Mensagem: Limite refeições fora a 1-2x por semana.
  Dica: Leve marmita para o trabalho pelo menos 3x por semana

  Regra 2: Estabelecer teto para lazer
  Categoria: Lazer
  Ação: lim

## 4. Função para Gerar Recomendações

In [11]:
def calcular_gasto_medio_categoria(user_id, categoria, transacoes_df):
    """
    Calcula o gasto médio mensal de um usuário em uma categoria.
    """
    user_transacoes = transacoes_df[
        (transacoes_df['user_id'] == user_id) & 
        (transacoes_df['categoria'] == categoria)
    ]
    
    if len(user_transacoes) == 0:
        return 0.0
    
    # Agrupar por mês e calcular média
    gasto_mensal = user_transacoes.groupby('mes')['valor'].sum()
    return gasto_mensal.mean()


def gerar_recomendacoes(user_id, cluster, transacoes_df, regras=REGRAS_RECOMENDACAO):
    """
    Gera recomendações personalizadas para um usuário baseado em seu cluster.
    
    Parâmetros:
        user_id: ID do usuário
        cluster: Número do cluster (0-3)
        transacoes_df: DataFrame com transações
        regras: Dicionário com regras de recomendação
    
    Retorna:
        Lista com 2 recomendações personalizadas
    """
    if cluster not in regras:
        raise ValueError(f"Cluster {cluster} não encontrado nas regras")
    
    cluster_regras = regras[cluster]
    recomendacoes = []
    
    for regra in cluster_regras['regras']:
        # Calcular gasto atual do usuário na categoria
        gasto_atual = calcular_gasto_medio_categoria(
            user_id, 
            regra['categoria'], 
            transacoes_df
        )
        
        # Calcular economia potencial
        economia_potencial = gasto_atual * regra['percentual']
        
        recomendacao = {
            'id': regra['id'],
            'titulo': regra['titulo'],
            'categoria': regra['categoria'],
            'acao': regra['acao'],
            'percentual_reducao': regra['percentual'],
            'gasto_atual': round(gasto_atual, 2),
            'economia_potencial': round(economia_potencial, 2),
            'mensagem': regra['mensagem'],
            'dica': regra['dica']
        }
        
        recomendacoes.append(recomendacao)
    
    return {
        'user_id': user_id,
        'cluster': cluster,
        'cluster_nome': cluster_regras['nome'],
        'prioridade': cluster_regras['prioridade'],
        'recomendacoes': recomendacoes,
        'economia_total': sum(r['economia_potencial'] for r in recomendacoes)
    }

print("Função gerar_recomendacoes() definida!")

Função gerar_recomendacoes() definida!


In [12]:
# Testar função com um usuário de cada cluster
print("\n" + "="*70)
print("TESTE: RECOMENDAÇÕES PARA 1 USUÁRIO DE CADA CLUSTER")
print("="*70)

for cluster_id in range(4):
    # Pegar primeiro usuário do cluster
    user_id = usuarios_clustered[usuarios_clustered['cluster'] == cluster_id]['user_id'].iloc[0]
    
    # Gerar recomendações
    resultado = gerar_recomendacoes(user_id, cluster_id, gastos)
    
    print(f"\n### {resultado['cluster_nome']} (Prioridade: {resultado['prioridade']})")
    print(f"Usuário: {resultado['user_id']}")
    print("-"*50)
    
    for i, rec in enumerate(resultado['recomendacoes'], 1):
        print(f"\n  Recomendação {i}: {rec['titulo']}")
        print(f"  Gasto atual: R$ {rec['gasto_atual']:.2f}/mês")
        print(f"  Economia potencial: R$ {rec['economia_potencial']:.2f}/mês ({rec['percentual_reducao']*100:.0f}%)")
    
    print(f"\n  >>> ECONOMIA TOTAL: R$ {resultado['economia_total']:.2f}/mês")


TESTE: RECOMENDAÇÕES PARA 1 USUÁRIO DE CADA CLUSTER

### Endividados Severos (Prioridade: CRÍTICA)
Usuário: user_0002
--------------------------------------------------

  Recomendação 1: Cortar refeições fora de casa
  Gasto atual: R$ 737.54/mês
  Economia potencial: R$ 516.28/mês (70%)

  Recomendação 2: Suspender compras de vestuário
  Gasto atual: R$ 362.20/mês
  Economia potencial: R$ 325.98/mês (90%)

  >>> ECONOMIA TOTAL: R$ 842.26/mês

### Em Alerta (Prioridade: MODERADA)
Usuário: user_0004
--------------------------------------------------

  Recomendação 1: Reduzir refeições fora de casa
  Gasto atual: R$ 342.10/mês
  Economia potencial: R$ 136.84/mês (40%)

  Recomendação 2: Estabelecer teto para lazer
  Gasto atual: R$ 89.54/mês
  Economia potencial: R$ 31.34/mês (35%)

  >>> ECONOMIA TOTAL: R$ 168.18/mês

### Endividados Moderados (Prioridade: ALTA)
Usuário: user_0008
--------------------------------------------------

  Recomendação 1: Reduzir significativamente refeiçõe

## 5. Salvar Regras em JSON

In [13]:
# Preparar regras para salvar (sem funções, apenas dados)
regras_para_salvar = {
    'versao': '1.0',
    'data_criacao': '2026-01-26',
    'descricao': 'Regras de recomendação de economia por cluster',
    'total_regras': 8,
    'clusters': REGRAS_RECOMENDACAO
}

# Salvar JSON
os.makedirs('../models', exist_ok=True)
caminho_json = '../models/recomendacoes_regras.json'

with open(caminho_json, 'w', encoding='utf-8') as f:
    json.dump(regras_para_salvar, f, ensure_ascii=False, indent=2)

print(f"Regras salvas em: {caminho_json}")

Regras salvas em: ../models/recomendacoes_regras.json


In [14]:
# Verificar arquivo salvo
with open(caminho_json, 'r', encoding='utf-8') as f:
    regras_carregadas = json.load(f)

print("Arquivo JSON carregado com sucesso!")
print(f"Versão: {regras_carregadas['versao']}")
print(f"Total de regras: {regras_carregadas['total_regras']}")
print(f"Clusters: {list(regras_carregadas['clusters'].keys())}")

Arquivo JSON carregado com sucesso!
Versão: 1.0
Total de regras: 8
Clusters: ['0', '1', '2', '3']


## 6. Resumo e Próximos Passos

In [15]:
# Calcular estatísticas das recomendações para todos os usuários
print("\n" + "="*70)
print("RESUMO: ECONOMIA POTENCIAL POR CLUSTER")
print("="*70)

economias_por_cluster = {}

for cluster_id in range(4):
    usuarios_cluster = usuarios_clustered[usuarios_clustered['cluster'] == cluster_id]
    economias = []
    
    for user_id in usuarios_cluster['user_id']:
        resultado = gerar_recomendacoes(user_id, cluster_id, gastos)
        economias.append(resultado['economia_total'])
    
    economias_por_cluster[cluster_id] = {
        'nome': NOMES_CLUSTERS[cluster_id],
        'num_usuarios': len(economias),
        'economia_media': np.mean(economias),
        'economia_total': np.sum(economias),
        'economia_min': np.min(economias),
        'economia_max': np.max(economias)
    }

# Exibir resultados
for cluster_id, stats in economias_por_cluster.items():
    print(f"\nCluster {cluster_id}: {stats['nome']}")
    print(f"  Usuários: {stats['num_usuarios']}")
    print(f"  Economia média: R$ {stats['economia_media']:.2f}/mês")
    print(f"  Economia total (cluster): R$ {stats['economia_total']:.2f}/mês")
    print(f"  Range: R$ {stats['economia_min']:.2f} - R$ {stats['economia_max']:.2f}")


RESUMO: ECONOMIA POTENCIAL POR CLUSTER



Cluster 0: Endividados Severos
  Usuários: 59
  Economia média: R$ 698.53/mês
  Economia total (cluster): R$ 41213.48/mês
  Range: R$ 369.12 - R$ 1431.23

Cluster 1: Em Alerta
  Usuários: 196
  Economia média: R$ 162.59/mês
  Economia total (cluster): R$ 31866.94/mês
  Range: R$ 75.90 - R$ 411.99

Cluster 2: Endividados Moderados
  Usuários: 167
  Economia média: R$ 320.13/mês
  Economia total (cluster): R$ 53461.81/mês
  Range: R$ 170.20 - R$ 660.75

Cluster 3: Poupadores
  Usuários: 78
  Economia média: R$ 120.90/mês
  Economia total (cluster): R$ 9430.02/mês
  Range: R$ 60.67 - R$ 252.17


In [16]:
# Calcular economia total projetada
economia_mensal_total = sum(e['economia_total'] for e in economias_por_cluster.values())
economia_anual_total = economia_mensal_total * 12

print("\n" + "="*70)
print("IMPACTO TOTAL PROJETADO")
print("="*70)
print(f"\nEconomia mensal total (500 usuários): R$ {economia_mensal_total:,.2f}")
print(f"Economia anual total: R$ {economia_anual_total:,.2f}")
print(f"Economia média por usuário: R$ {economia_mensal_total/500:.2f}/mês")


IMPACTO TOTAL PROJETADO

Economia mensal total (500 usuários): R$ 135,972.25
Economia anual total: R$ 1,631,667.00
Economia média por usuário: R$ 271.94/mês


In [17]:
# Checklist Dia 8
print("\n" + "="*70)
print("CHECKLIST DIA 8")
print("="*70)
print("\n[x] Carregar usuarios_clustered.csv e transacoes.csv")
print("[x] Analisar gastos por categoria e cluster")
print("[x] Definir 2 regras de recomendação por cluster (8 total)")
print("[x] Criar função gerar_recomendacoes(user_id, cluster)")
print("[x] Salvar regras em models/recomendacoes_regras.json")
print("[x] Calcular economia potencial por cluster")
print("\n>>> DIA 8 CONCLUÍDO!")
print("\nPróximo passo (Dia 9): Calcular economia detalhada e validar H1")


CHECKLIST DIA 8

[x] Carregar usuarios_clustered.csv e transacoes.csv
[x] Analisar gastos por categoria e cluster
[x] Definir 2 regras de recomendação por cluster (8 total)
[x] Criar função gerar_recomendacoes(user_id, cluster)
[x] Salvar regras em models/recomendacoes_regras.json
[x] Calcular economia potencial por cluster

>>> DIA 8 CONCLUÍDO!

Próximo passo (Dia 9): Calcular economia detalhada e validar H1
